[![AWS Data Wrangler](_static/logo.png "AWS Data Wrangler")](https://github.com/awslabs/aws-data-wrangler)

# 31 - OpenSearch

## Table of Contents
* [1. Create Indices](#2.-Create-Indices)
* [2. Write Indices](#3.-Write-Indices)
	* [2.1 Writing from JSON file](#2.1-Writing-from-JSON-file)
	* [2.2 Writing from CSV file](#2.2-Writing-from-CSV-file)
* [3. Search Indices](#1.-Search-Indices)
	* [3.1 Search by DSL](#1.1-Search-by-DSL)
	* [3.2 Search by SQL](#1.2-Search-by-SQL)
* [4. Delete Indices](#7.-Delete-Indices)


In [1]:
import awswrangler as wr


## Enter your domain endpoint:

In [2]:
client = wr.opensearch.connect(host='DOMAIN-ENDPOINT')

## 1. Create Indices

In [3]:
response = wr.opensearch.create_index(
     client=client,
     index="tutorials",
     mappings={
         "properties": {
              "id":  { "type" : "integer" },
              "id":  { "type" : "string" },
          }
     },
     settings={
             "index": {
                 "number_of_shards": 2
                 "number_of_replicas": 1
             }
      }
)


## 2. Write Indices

### 2.1 Write from JSON files

In [4]:
wr.opensearch.index_json(
       client=client,
       path='s3://awswrangler-opensearch/dataload/doc1.json',
       index='tutorials'
     )


### 2.2 Write from CSV files

In [5]:
wr.opensearch.index_csv(
       client=client,
       path='s3://awswrangler-opensearch/dataload/doc1.csv',
       index='tutorials'
     )


## 3. Search Indices

#### 3.1 Search by DSL

In [6]:
df = wr.opensearch.search(
         client=client,
         index='tutorials',
         search_body={
           "query": {
            "match_all": {
             }
           }
          }
     )


#### 3.1 Search by SQL

In [7]:
df = wr.opensearch.search_by_sql(
         client=client,
         sql_query='SELECT * FROM tutorials LIMIT 50'
     )


## 4. Delete Index

In [8]:
response = wr.opensearch.delete_index(
     client=client,
     index="tutorials",
 )